In [0]:
from pyspark.sql import functions as F

In [0]:
tran_inmobiliarias = _sqldf

# Seleccionar columnas a usar en el análisis

Este análisis se enfocará en anomalías de valor (valores muy altos, valorización muy alta en cortos periodos de tiempo), cantidad de transacciones en corto periodo de tiempo y predios con fecha de apertura diferentes.

In [0]:
tran_inmobiliarias = tran_inmobiliarias.select('MATRICULA', 'VALOR', 'FECHA_RADICA_TEXTO', 'FECHA_APERTURA_TEXTO')
tran_inmobiliarias.display()

# Estandarizar fecha de radicación

In [0]:
# 1. Columna auxiliar para el formato ISO YYYY-MM-DD
tran_inmobiliarias = tran_inmobiliarias.withColumn(
    'FECHA_RADICA_TEMP_ISO',
    F.when(
        F.col('FECHA_RADICA_TEXTO').rlike('^\\d{4}-\\d{2}-\\d{2}'),
        F.to_date(F.substring(F.col('FECHA_RADICA_TEXTO'), 1, 10), 'yyyy-MM-dd')
    ).otherwise(F.lit(None).cast('date')) # Devuelve NULL si no coincide
)

# 2. Columna auxiliar para el formato DD/MM/YYYY (4 dígitos de año)
# Usamos F.regexp_extract para extraer la fecha e ignorar cualquier texto o número inicial.
tran_inmobiliarias = tran_inmobiliarias.withColumn(
    'FECHA_RADICA_TEMP_DMY4',
    F.when(
        # Verifica que el patrón DD/MM/YYYY exista en algún lugar de la cadena
        F.col('FECHA_RADICA_TEXTO').rlike('\\d{2}/\\d{2}/\\d{4}'),
        # Extrae solo el grupo de la fecha y lo convierte
        F.to_date(
            F.regexp_extract(F.col('FECHA_RADICA_TEXTO'), '(\\d{2}/\\d{2}/\\d{4})', 1),
            'dd/MM/yyyy'
        )
    ).otherwise(F.lit(None).cast('date')) # Devuelve NULL si no coincide
)

# 3. Columna auxiliar para el formato DD/MM/YY (2 dígitos de año)
# Usamos F.regexp_extract para extraer la fecha e ignorar cualquier texto o número inicial,
# solucionando casos como '25841 25/08/22'.
tran_inmobiliarias = tran_inmobiliarias.withColumn(
    'FECHA_RADICA_TEMP_DMY2',
    F.when(
        # Verifica que el patrón DD/MM/YY exista en algún lugar de la cadena
        F.col('FECHA_RADICA_TEXTO').rlike('\\d{2}/\\d{2}/\\d{2}'),
        # Extrae solo el grupo de la fecha y lo convierte
        F.to_date(
            F.regexp_extract(F.col('FECHA_RADICA_TEXTO'), '(\\d{2}/\\d{2}/\\d{2})', 1),
            'dd/MM/yy'
        )
    ).otherwise(F.lit(None).cast('date')) # Devuelve NULL si no coincide
)

# 4. Consolidar las fechas: Usamos F.coalesce para tomar el primer valor de fecha no nulo
# y SOBRESCRIBIMOS la columna original 'FECHA_RADICA_TEXTO'.
tran_inmobiliarias = tran_inmobiliarias.withColumn(
    'FECHA_RADICA_TEXTO', # Reemplaza la columna original (ahora de tipo Date)
    F.coalesce(
        F.col('FECHA_RADICA_TEMP_ISO'),
        F.col('FECHA_RADICA_TEMP_DMY4'),
        F.col('FECHA_RADICA_TEMP_DMY2')
    )
)

# 5. Eliminar las columnas auxiliares temporales.
tran_inmobiliarias = tran_inmobiliarias.drop(
    'FECHA_RADICA_TEMP_ISO',
    'FECHA_RADICA_TEMP_DMY4',
    'FECHA_RADICA_TEMP_DMY2'
)

# Mostrar el resultado final con la columna de fecha ya convertida
display(tran_inmobiliarias)

# Estandarizar fecha de apertura

In [0]:
# 1. Columna auxiliar para el formato ISO YYYY-MM-DD
tran_inmobiliarias = tran_inmobiliarias.withColumn(
    'FECHA_APERTURA_TEMP_ISO',
    F.when(
        F.col('FECHA_APERTURA_TEXTO').rlike('^\\d{4}-\\d{2}-\\d{2}'),
        F.to_date(F.substring(F.col('FECHA_APERTURA_TEXTO'), 1, 10), 'yyyy-MM-dd')
    ).otherwise(F.lit(None).cast('date')) # Devuelve NULL si no coincide
)

# 2. Columna auxiliar para el formato DD/MM/YYYY (4 dígitos de año)
# Usamos F.regexp_extract para extraer la fecha e ignorar cualquier texto o número inicial.
tran_inmobiliarias = tran_inmobiliarias.withColumn(
    'FECHA_APERTURA_TEMP_DMY4',
    F.when(
        # Verifica que el patrón DD/MM/YYYY exista en algún lugar de la cadena
        F.col('FECHA_APERTURA_TEXTO').rlike('\\d{2}/\\d{2}/\\d{4}'),
        # Extrae solo el grupo de la fecha y lo convierte
        F.to_date(
            F.regexp_extract(F.col('FECHA_APERTURA_TEXTO'), '(\\d{2}/\\d{2}/\\d{4})', 1),
            'dd/MM/yyyy'
        )
    ).otherwise(F.lit(None).cast('date')) # Devuelve NULL si no coincide
)

# 3. Columna auxiliar para el formato DD/MM/YY (2 dígitos de año)
# Usamos F.regexp_extract para extraer la fecha e ignorar cualquier texto o número inicial,
# solucionando casos como '25841 25/08/22'.
tran_inmobiliarias = tran_inmobiliarias.withColumn(
    'FECHA_APERTURA_TEMP_DMY2',
    F.when(
        # Verifica que el patrón DD/MM/YY exista en algún lugar de la cadena
        F.col('FECHA_APERTURA_TEXTO').rlike('\\d{2}/\\d{2}/\\d{2}'),
        # Extrae solo el grupo de la fecha y lo convierte
        F.to_date(
            F.regexp_extract(F.col('FECHA_APERTURA_TEXTO'), '(\\d{2}/\\d{2}/\\d{2})', 1),
            'dd/MM/yy'
        )
    ).otherwise(F.lit(None).cast('date')) # Devuelve NULL si no coincide
)

# 4. Consolidar las fechas: Usamos F.coalesce para tomar el primer valor de fecha no nulo
# y SOBRESCRIBIMOS la columna original 'FECHA_RADICA_TEXTO'.
tran_inmobiliarias = tran_inmobiliarias.withColumn(
    'FECHA_APERTURA_TEXTO', # Reemplaza la columna original (ahora de tipo Date)
    F.coalesce(
        F.col('FECHA_APERTURA_TEMP_ISO'),
        F.col('FECHA_APERTURA_TEMP_DMY4'),
        F.col('FECHA_APERTURA_TEMP_DMY2')
    )
)

# 5. Eliminar las columnas auxiliares temporales.
tran_inmobiliarias = tran_inmobiliarias.drop(
    'FECHA_APERTURA_TEMP_ISO',
    'FECHA_APERTURA_TEMP_DMY4',
    'FECHA_APERTURA_TEMP_DMY2'
)

# Mostrar el resultado final con la columna de fecha ya convertida
display(tran_inmobiliarias)

In [0]:
tran_inmobiliarias.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("data_modelo_anomalias_transaccionales")